In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 데이터 로딩 ###
dataset = pd.read_csv('/content/drive/MyDrive/빅데이터 팀플/dementia_analytics/data_processing/dataset_05/dataset_with_label_fill_user_mean.csv')

data_output = dataset[['DIAG_NM']]
data_input = dataset[['sleep_breath_average', 'sleep_hr_average', 'sleep_hr_lowest', 'sleep_deep', 'sleep_rem', 'activity_cal_total', 'sleep_awake', 'activity_steps', 'activity_total', 'sleep_duration', 'activity_daily_movement']]

### train test split ###
from sklearn.model_selection import train_test_split

train_input, test_input, train_output, test_output = train_test_split(data_input, data_output, test_size=0.2, shuffle=True, stratify=data_output, random_state=42)

### test_output 레이블 인코딩 ###
from sklearn.preprocessing import LabelEncoder

# 라벨인코더 선언 및 Fitting
# CN : 0, Dem : 1, MCI : 2
items = ['CN', 'Dem', 'MCI']
encoder = LabelEncoder()
encoder = encoder.fit(items)

test_output['DIAG_NM'] = encoder.transform(test_output['DIAG_NM'])

In [ ]:
test_output.shape

(2278, 1)

In [ ]:
loaded_scaler = joblib.load('/content/drive/MyDrive/빅데이터 팀플/model/final_lgbm_scaler.pkl')
loaded_model = joblib.load('/content/drive/MyDrive/빅데이터 팀플/model/final_lgbm_model.pkl')
for k in range(2278):
  pred = loaded_model.predict(loaded_scaler.transform(test_input[k:k+1]))
  if k % 100 == 0: # 몇 개만 출력해 확인
    print(k, "번째:")
    print("예측:", pred)
    print("실제:", list(test_output[k:k+1]['DIAG_NM']))
    print()
    # print(test_input.head())
    # print(test_output.head())

0 번째:
예측: [0]
실제: [0]

100 번째:
예측: [0]
실제: [2]

200 번째:
예측: [2]
실제: [2]

300 번째:
예측: [0]
실제: [0]

400 번째:
예측: [0]
실제: [0]

500 번째:
예측: [0]
실제: [0]

600 번째:
예측: [2]
실제: [2]

700 번째:
예측: [0]
실제: [0]

800 번째:
예측: [2]
실제: [2]

900 번째:
예측: [0]
실제: [0]

1000 번째:
예측: [0]
실제: [0]

1100 번째:
예측: [2]
실제: [0]

1200 번째:
예측: [0]
실제: [2]

1300 번째:
예측: [1]
실제: [1]

1400 번째:
예측: [0]
실제: [0]

1500 번째:
예측: [2]
실제: [2]

1600 번째:
예측: [2]
실제: [2]

1700 번째:
예측: [0]
실제: [0]

1800 번째:
예측: [0]
실제: [0]

1900 번째:
예측: [2]
실제: [2]

2000 번째:
예측: [2]
실제: [2]

2100 번째:
예측: [2]
실제: [0]

2200 번째:
예측: [0]
실제: [0]



In [ ]:
test_output[k:k+1]['DIAG_NM'] == 2

7592    False
Name: DIAG_NM, dtype: bool

In [ ]:
loaded_scaler = joblib.load('/content/drive/MyDrive/빅데이터 팀플/model/final_lgbm_scaler.pkl')
loaded_model = joblib.load('/content/drive/MyDrive/빅데이터 팀플/model/final_lgbm_model.pkl')

cnt02 = 0
cnt01 = 0
cnt12 = 0

for k in range(2278):
  pred = loaded_model.predict(loaded_scaler.transform(test_input[k:k+1]))
  actual = list(test_output[k:k+1]['DIAG_NM'])
  if actual != pred: # 몇 개만 출력해 확인
    if (pred+actual)==2:
      cnt02+=1
    elif (pred+actual)==1:
      cnt01+=1
    else:
      cnt12+=1

print("정상과 경도인지장애 혼동 :", cnt02)
print("정상과 치매 혼동 :", cnt01)
print("경도인지장애와 치매 혼동 :", cnt12)

정상과 경도인지장애 혼동 : 436
정상과 치매 혼동 : 56
경도인지장애와 치매 혼동 : 26


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

test_pred = loaded_model.predict(loaded_scaler.transform(test_input))

# 정확도 계산
accuracy = accuracy_score(test_output, test_pred)
print("Accuracy:", accuracy)

# 정밀도 계산
precision = precision_score(test_output, test_pred, average='macro')
print("Precision:", precision)

# 재현율 계산
recall = recall_score(test_output, test_pred, average='macro')
print("Recall:", recall)

# F1 스코어 계산
f1 = f1_score(test_output, test_pred, average='macro')
print("F1 Score:", f1)

Accuracy: 0.7726075504828798
Precision: 0.7308847434425226
Recall: 0.7808662772477639
F1 Score: 0.7526889999748554
